In [1]:
import pandas as pd
import numpy as np
import typing as tp

from datetime import datetime, timedelta
from dateutil.parser import parse
from scipy.stats import levy_stable

In [2]:
# Import csv data for last <= 90 days of priceCumulative values from csv/data-1624665179.csv
DATA_FILENAME = "data-1625069716"
df_raw = pd.read_csv(f"csv/{DATA_FILENAME}.csv")
df_raw

,result,table,_start,_stop,_time,_value,_field,_measurement,id,influx-sushi,type,token0_name,token1_name
0,_result,0,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-04-18 18:50:49+00:00,6.086160e+41,price1Cumulative,mem,Sushiswap: AAVE / WETH,ingest-data-frame,metrics-hourly,NaN,NaN
1,_result,0,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-04-18 19:42:33+00:00,6.087104e+41,price1Cumulative,mem,Sushiswap: AAVE / WETH,ingest-data-frame,metrics-hourly,NaN,NaN
2,_result,0,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-04-18 20:29:26+00:00,6.087963e+41,price1Cumulative,mem,Sushiswap: AAVE / WETH,ingest-data-frame,metrics-hourly,NaN,NaN
3,_result,0,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-04-18 20:48:38+00:00,6.088319e+41,price1Cumulative,mem,Sushiswap: AAVE / WETH,ingest-data-frame,metrics-hourly,NaN,NaN
4,_result,0,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-04-18 21:02:36+00:00,6.088578e+41,price1Cumulative,mem,Sushiswap: AAVE / WETH,ingest-data-frame,metrics-hourly,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
198881,_result,57,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-06-30 15:23:52+00:00,1.585105e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC
198882,_result,57,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-06-30 15:36:46+00:00,1.585124e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC
198883,_result,57,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-06-30 15:48:16+00:00,1.585141e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC
198884,_result,57,2021-04-01 16:15:11.327820+00:00,2021-06-30 16:15:11.327820+00:00,2021-06-30 15:58:36+00:00,1.585156e+50,price0Cumulative,mem,Sushiswap: WETH / USDC,ingest-data-frame,metrics-hourly,WETH,USDC


In [3]:
# Filter for cols we care about: [id, _time, _field, _value] and sort by time
df = df_raw.filter(items=['id', '_time', '_field', '_value'])
df = df.sort_values(by='_time', ignore_index=True)
df

,id,_time,_field,_value
0,Sushiswap: COMP / WETH,2021-04-18 17:47:35+00:00,price0Cumulative,2.528080e+40
1,Sushiswap: COMP / WETH,2021-04-18 17:47:35+00:00,price1Cumulative,4.148446e+41
2,Sushiswap: UNI / WETH,2021-04-18 18:31:43+00:00,price1Cumulative,1.109690e+43
3,Sushiswap: UNI / WETH,2021-04-18 18:31:43+00:00,price0Cumulative,9.936822e+38
4,Sushiswap: ALPHA / WETH,2021-04-18 18:42:40+00:00,price0Cumulative,4.329733e+37
...,...,...,...,...
198881,Sushiswap: WETH / DAI,2021-06-30 16:09:58+00:00,price0Cumulative,1.596079e+38
198882,Sushiswap: WETH / DAI,2021-06-30 16:09:58+00:00,price1Cumulative,1.906537e+44
198883,Sushiswap: WETH / USDC,2021-06-30 16:09:58+00:00,price0Cumulative,1.585173e+50
198884,Sushiswap: LINK / WETH,2021-06-30 16:10:58+00:00,price0Cumulative,2.476750e+39


In [4]:
# Examine stats for pairs: WETH / DAI (less volatile) ...
df_weth_dai = df[(df['id'] == "Sushiswap: WETH / DAI") & (df['_field'] == "price1Cumulative")]
df_weth_dai

,id,_time,_field,_value
81362,Sushiswap: WETH / DAI,2021-05-19 20:47:17+00:00,price1Cumulative,1.459467e+44
81397,Sushiswap: WETH / DAI,2021-05-19 21:07:02+00:00,price1Cumulative,1.459625e+44
81419,Sushiswap: WETH / DAI,2021-05-19 21:17:59+00:00,price1Cumulative,1.459719e+44
81448,Sushiswap: WETH / DAI,2021-05-19 21:28:19+00:00,price1Cumulative,1.459791e+44
81489,Sushiswap: WETH / DAI,2021-05-19 21:40:59+00:00,price1Cumulative,1.459890e+44
...,...,...,...,...
198816,Sushiswap: WETH / DAI,2021-06-30 15:25:30+00:00,price1Cumulative,1.906246e+44
198833,Sushiswap: WETH / DAI,2021-06-30 15:34:33+00:00,price1Cumulative,1.906306e+44
198855,Sushiswap: WETH / DAI,2021-06-30 15:47:19+00:00,price1Cumulative,1.906389e+44
198866,Sushiswap: WETH / DAI,2021-06-30 15:57:01+00:00,price1Cumulative,1.906453e+44


In [5]:
# ... and WETH / USDC (less volatile, but we have more data than above) ...
df_weth_usdc = df[(df['id'] == "Sushiswap: WETH / USDC") & (df['_field'] == "price1Cumulative")]
df_weth_usdc

,id,_time,_field,_value
20,Sushiswap: WETH / USDC,2021-04-18 18:52:52+00:00,price1Cumulative,1.029267e+32
39,Sushiswap: WETH / USDC,2021-04-18 18:56:00+00:00,price1Cumulative,1.029289e+32
58,Sushiswap: WETH / USDC,2021-04-18 19:55:50+00:00,price1Cumulative,1.029708e+32
90,Sushiswap: WETH / USDC,2021-04-18 20:30:01+00:00,price1Cumulative,1.029945e+32
109,Sushiswap: WETH / USDC,2021-04-18 20:40:49+00:00,price1Cumulative,1.030020e+32
...,...,...,...,...
198812,Sushiswap: WETH / USDC,2021-06-30 15:23:52+00:00,price1Cumulative,1.909376e+32
198839,Sushiswap: WETH / USDC,2021-06-30 15:36:46+00:00,price1Cumulative,1.909460e+32
198861,Sushiswap: WETH / USDC,2021-06-30 15:48:16+00:00,price1Cumulative,1.909535e+32
198873,Sushiswap: WETH / USDC,2021-06-30 15:58:36+00:00,price1Cumulative,1.909603e+32


In [6]:
# ... and ALCX / WETH (more volatile)
df_alcx_weth = df[(df['id'] == "Sushiswap: ALCX / WETH") & (df['_field'] == "price1Cumulative")]
df_alcx_weth

,id,_time,_field,_value
9,Sushiswap: ALCX / WETH,2021-04-18 18:50:49+00:00,price1Cumulative,1.831283e+40
35,Sushiswap: ALCX / WETH,2021-04-18 18:55:04+00:00,price1Cumulative,1.831382e+40
57,Sushiswap: ALCX / WETH,2021-04-18 19:55:02+00:00,price1Cumulative,1.832779e+40
87,Sushiswap: ALCX / WETH,2021-04-18 20:28:40+00:00,price1Cumulative,1.833563e+40
103,Sushiswap: ALCX / WETH,2021-04-18 20:38:26+00:00,price1Cumulative,1.833792e+40
...,...,...,...,...
198761,Sushiswap: ALCX / WETH,2021-06-30 15:02:17+00:00,price1Cumulative,2.987192e+40
198790,Sushiswap: ALCX / WETH,2021-06-30 15:14:29+00:00,price1Cumulative,2.987252e+40
198808,Sushiswap: ALCX / WETH,2021-06-30 15:23:19+00:00,price1Cumulative,2.987296e+40
198837,Sushiswap: ALCX / WETH,2021-06-30 15:36:26+00:00,price1Cumulative,2.987360e+40


In [7]:
# ... and UNI / WETH (mid volatile)
df_uni_weth = df[(df['id'] == "Sushiswap: UNI / WETH") & (df['_field'] == "price0Cumulative")]
df_uni_weth

,id,_time,_field,_value
3,Sushiswap: UNI / WETH,2021-04-18 18:31:43+00:00,price0Cumulative,9.936822e+38
33,Sushiswap: UNI / WETH,2021-04-18 18:55:04+00:00,price0Cumulative,9.937873e+38
67,Sushiswap: UNI / WETH,2021-04-18 19:58:10+00:00,price0Cumulative,9.940690e+38
72,Sushiswap: UNI / WETH,2021-04-18 20:13:22+00:00,price0Cumulative,9.941360e+38
130,Sushiswap: UNI / WETH,2021-04-18 20:51:09+00:00,price0Cumulative,9.943018e+38
...,...,...,...,...
198292,Sushiswap: UNI / WETH,2021-06-30 10:44:44+00:00,price0Cumulative,1.340872e+39
198353,Sushiswap: UNI / WETH,2021-06-30 11:11:53+00:00,price0Cumulative,1.340943e+39
198534,Sushiswap: UNI / WETH,2021-06-30 13:06:27+00:00,price0Cumulative,1.341243e+39
198687,Sushiswap: UNI / WETH,2021-06-30 14:26:26+00:00,price0Cumulative,1.341451e+39


In [8]:
# TODO:
#  1. Compute the TWAP from PC value in df [x]
#  2. Fit Levy stable with scipy built in MLE code [x]
#  3. Report Levy stable params fit (a, b, mu, sig) [x]
#  4. Calc [e**(mu*t + sig * (t/a)**(1/a) * F^{-1}(1-alpha)) - 1] (VaR * d**m normalized for imbalance) for diff t vals
#  5. See how large/small C_p = 4. can be for various t, alpha combos

In [9]:
# Define some functions to calc twap
PC_RESOLUTION = 112

def compute_amount_out(twap_112: np.ndarray, amount_in: int) -> np.ndarray:
    rshift = np.vectorize(lambda x: int(x * amount_in) >> PC_RESOLUTION)
    return rshift(twap_112)

def get_twap(pc: pd.DataFrame, window: int, amount_in: int) -> pd.DataFrame:
    dp = pc.filter(items=['_value'])\
        .rolling(window=window)\
        .apply(lambda w: w[-1] - w[0], raw=True)
    
    # for time, need to map to timestamp first then apply delta
    dt = pc.filter(items=['_time'])\
        .applymap(parse)\
        .applymap(datetime.timestamp)\
        .rolling(window=window)\
        .apply(lambda w: w[-1] - w[0], raw=True)

    # with NaNs filtered out
    twap_112 = (dp['_value'] / dt['_time']).to_numpy()
    twap_112 = twap_112[np.logical_not(np.isnan(twap_112))]
    twaps = compute_amount_out(twap_112, amount_in)
    
    # window close timestamps
    t = pc.filter(items=['_time'])\
        .applymap(parse)\
        .applymap(datetime.timestamp)\
        .rolling(window=window)\
        .apply(lambda w: w[-1], raw=True)
    ts = t['_time'].to_numpy()
    ts = ts[np.logical_not(np.isnan(ts))]
    
    df = pd.DataFrame(data=[ts, twaps]).T
    df.columns = ['timestamp', 'twap']
    
    # filter out any twaps that are less than or equal to 0
    df = df[df['twap'] > 0]
    return df

In [10]:
# 1. Compute the TWAP from PC value in df
WINDOW = 6
UNIT_WETH = 1e18

df_weth_dai_twap = get_twap(pc=df_weth_dai, window=WINDOW, amount_in=UNIT_WETH)
df_weth_usdc_twap = get_twap(pc=df_weth_usdc, window=WINDOW, amount_in=UNIT_WETH)
df_alcx_weth_twap = get_twap(pc=df_alcx_weth, window=WINDOW, amount_in=UNIT_WETH)
df_uni_weth_twap = get_twap(pc=df_uni_weth, window=WINDOW, amount_in=UNIT_WETH)

In [11]:
df_weth_dai_twap

,timestamp,twap
0,1.62146e+09,2530192649420577177600
1,1.62146e+09,2526661690450846416896
2,1.62146e+09,2504822078413549862912
3,1.62146e+09,2580894455288732057600
4,1.62146e+09,2610532884887874043904
...,...,...
5233,1.62507e+09,2121012659322759151616
5234,1.62507e+09,2116564588953760432128
5235,1.62507e+09,2111598742662454706176
5236,1.62507e+09,2106627667512922472448


In [11]:
df_weth_dai_twap['twap'] / UNIT_WETH

0       2823.28
1       2807.42
2       2795.74
3       2778.51
4       2767.56
         ...   
3744    1832.47
3745    1825.07
3746    1817.09
3747    1810.49
3748     1810.8
Name: twap, Length: 3749, dtype: object

In [12]:
df_weth_usdc_twap

,timestamp,twap
0,1.618779e+09,2.239566e+09
1,1.618780e+09,2.240544e+09
2,1.618780e+09,2.238170e+09
3,1.618781e+09,2.250825e+09
4,1.618782e+09,2.256557e+09
...,...,...
9175,1.625067e+09,2.122203e+09
9176,1.625067e+09,2.115873e+09
9177,1.625068e+09,2.111369e+09
9178,1.625069e+09,2.106606e+09


In [13]:
# Export twaps to csv to analyze in mathematica as well
df_weth_dai_twap.to_csv(f"csv/{DATA_FILENAME}_weth-dai-twap.csv")
df_weth_usdc_twap.to_csv(f"csv/{DATA_FILENAME}_weth-usdc-twap.csv")
df_alcx_weth_twap.to_csv(f"csv/{DATA_FILENAME}_alcx-weth-twap.csv")
df_uni_weth_twap.to_csv(f"csv/{DATA_FILENAME}_uni-weth-twap.csv")

In [12]:
# Define some functions to fit TWAP to log stable
def get_rs(twap: pd.DataFrame) -> np.ndarray:
    sample = twap['twap'].to_numpy()
    return [
        np.log(sample[i]/sample[i-1])
        for i in range(1, len(sample), 1)
    ]

def fit_to_log_stable(twap: pd.DataFrame) -> tp.Tuple[float]:
    rs = get_rs(twap)
    return levy_stable.fit(rs)

def get_params(twap: pd.DataFrame, period: int) -> pd.DataFrame:
    a, b, loc, scale = fit_to_log_stable(twap)

    # transform to mu, sig
    mu = loc / period
    sig = scale / (period/a)**(1/a)
    
    df = pd.DataFrame(data=[a, b, mu, sig]).T
    df.columns = ['a', 'b', 'mu', 'sig']
    return df

In [13]:
# 2. Fit Levy stable with scipy built in MLE code
PERIOD = 600 # 10 min

# NOTE: this is taking too long ... what's a way around this while still using scipy?
# Could wrap C++ code from Nolan instead ...
# TODO: export df_twap to csv and use mathematica to check fitting is ok (is scipy issue)
# df_weth_dai_params = get_params(twap=df_weth_dai_twap, period=PERIOD)
# df_alcx_weth_params = get_params(twap=df_alcx_weth_twap, period=PERIOD)
# df_uni_weth_params = get_params(twap=df_uni_weth_twap, period=PERIOD)

In [14]:
# 3. Report Levy stable params fit (a, b, mu, sig)
# df_weth_dai_params

NameError: name 'df_weth_dai_params' is not defined